In [1]:
import numpy as np
import pandas as pd


In [2]:
import snscrape.modules.twitter as tw

f = open('em.txt','w',encoding='utf-8')

#Save Twitter data to external file
#Possible outputs: url, date, content, id, username, outlinks, outlinksss, tcooutlinks, tcooutlinksss
for tweet in tw.TwitterSearchScraper(query="(from:elonmusk) since:2020-08-01 until:2020-11-01").get_items():
    date_str = tweet.date.strftime("%Y-%m-%d %H:%M:%S%z")
    date_str = date_str[:-2] + ":" + date_str[-2:]
    #f.write(date_str + "|" + tweet.content + "\n")
    f.write(date_str + "|" + tweet.rawContent + "\n")
f.close()

In [3]:
from datetime import datetime as dt
import pytz

#Read Twitter data into Python
em = []
dates = []
f = open("em.txt", "r", encoding="utf-8")
for l in f:
    line = l.split("|")
    date_str = line[0]#+"+00:00"
    try:
        date_time = dt.fromisoformat(date_str)
        date_time = date_time.astimezone(pytz.timezone("US/Eastern"))
        line[0] = date_time
        line[1] = line[1][:-1]
        em.append(line)
        dates.append(date_time.date())
    except:
        em[-1][1] += " "+l[:-1]
f.close()

em = pd.DataFrame(data=em , columns=['Time','Tweet'])
em['Date'] = dates
#em['Date'] = em['Date'].astype(str)
print(em)

                         Time  \
0   2020-10-31 19:34:19-04:00   
1   2020-10-31 18:09:40-04:00   
2   2020-10-31 18:04:30-04:00   
3   2020-10-31 18:00:41-04:00   
4   2020-10-31 17:59:44-04:00   
..                        ...   
760 2020-08-02 00:49:11-04:00   
761 2020-08-01 04:15:54-04:00   
762 2020-08-01 02:47:18-04:00   
763 2020-07-31 23:57:37-04:00   
764 2020-07-31 23:51:54-04:00   

                                                 Tweet        Date  
0                       @RGVaerialphotos Lord of the …  2020-10-31  
1    @TGMetsFan98 @KlotzAdam @NASASpaceflight @Erda...  2020-10-31  
2      @flcnhvy @NASASpaceflight @Erdayastronaut Minor  2020-10-31  
3    @NASASpaceflight @Erdayastronaut But, a RUD ri...  2020-10-31  
4    @NASASpaceflight @Erdayastronaut Stable, contr...  2020-10-31  
..                                                 ...         ...  
760  @meier1028 @SpaceX @Space_Station @AstroBehnke...  2020-08-02  
761  @engineeringvids The simplest solution is not 

In [4]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#The compound score is computed by summing the valence scores of each word in the lexicon, 
#adjusted according to the rules, and then normalized to be between -1 (most extreme negative) 
#and +1 (most extreme positive). This is the most useful metric if you want a single unidimensional 
#measure of sentiment for a given sentence. Calling it a 'normalized, weighted composite score' is accurate.
#It is also useful for researchers who would like to set standardized thresholds for classifying sentences 
#as either positive, neutral, or negative. Typical threshold values (used in the literature cited on this page) are:
#    positive sentiment: compound score >= 0.05
#    neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
#    negative sentiment: compound score <= -0.05
#The pos, neu, and neg scores are ratios for proportions of text that fall in each category (so these should all 
#add up to be 1... or close to it with float operation). These are the most useful metrics if you want 
#multidimensional measures of sentiment for a given sentence.

sentiment = []
analyzer = SentimentIntensityAnalyzer()
for tweet in em.Tweet:
    vs = analyzer.polarity_scores(tweet)
    sentiment.append(vs["compound"])
    
em['Sentiment'] = sentiment
print(em)

                         Time  \
0   2020-10-31 19:34:19-04:00   
1   2020-10-31 18:09:40-04:00   
2   2020-10-31 18:04:30-04:00   
3   2020-10-31 18:00:41-04:00   
4   2020-10-31 17:59:44-04:00   
..                        ...   
760 2020-08-02 00:49:11-04:00   
761 2020-08-01 04:15:54-04:00   
762 2020-08-01 02:47:18-04:00   
763 2020-07-31 23:57:37-04:00   
764 2020-07-31 23:51:54-04:00   

                                                 Tweet        Date  Sentiment  
0                       @RGVaerialphotos Lord of the …  2020-10-31     0.0000  
1    @TGMetsFan98 @KlotzAdam @NASASpaceflight @Erda...  2020-10-31    -0.1531  
2      @flcnhvy @NASASpaceflight @Erdayastronaut Minor  2020-10-31     0.0000  
3    @NASASpaceflight @Erdayastronaut But, a RUD ri...  2020-10-31     0.4235  
4    @NASASpaceflight @Erdayastronaut Stable, contr...  2020-10-31     0.8858  
..                                                 ...         ...        ...  
760  @meier1028 @SpaceX @Space_Station @Ast

In [5]:
import yfinance
from datetime import datetime

#Load daily Tesla Adjusted Close data and compute log returns
tesla = yfinance.download("TSLA",datetime(2020,8,1),datetime(2020,10,1)) 
tesla['log_ret'] = np.log(tesla['Adj Close'].shift(-1)) - np.log(tesla['Adj Close'])

#Construct average sentiment on each day
prev_d = datetime(2020,1,1).date()
avg_sentiment = []
for date in tesla.index.values:
    d = pd.to_datetime(date).date()
    #tesla.loc[tesla.index == date] = np.mean(em.loc[(em.Date <= d) & (em.Date > prev_d)].Sentiment)
    avg = np.mean(em.loc[(em.Date <= d) & (em.Date > prev_d)].Sentiment)
    if np.isnan(avg):
        avg_sentiment.append(0)
    else:
        avg_sentiment.append(avg)
    prev_d = d
tesla['sentiment'] = avg_sentiment

#Drop final row
tesla = tesla.iloc[:-1,]
print(tesla)

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2020-08-03   96.613335  100.653999   96.292000   99.000000   99.000000   
2020-08-04   99.667336  101.827332   97.466667   99.133331   99.133331   
2020-08-05   99.532669   99.989334   97.887337   99.001335   99.001335   
2020-08-06   99.388664  101.153999   98.484001   99.305336   99.305336   
2020-08-07   99.969330   99.983330   94.334000   96.847336   96.847336   
2020-08-10   96.533333   97.166664   92.389336   94.571335   94.571335   
2020-08-11   93.066666   94.666664   91.000000   91.625999   91.625999   
2020-08-12   98.000000  105.666664   95.666664  103.650665  103.650665   
2020-08-13  107.400002  110.078667  104.484001  108.066666  108.066666   
2020-08-14  110.999336  111.253334  108.442665  110.047333  110.047333   
2020-08-17  111.800003  123.057335  111.522

In [22]:
# Compare linear models
from sklearn.linear_model import LinearRegression

## 1-day lag returns
r = tesla['log_ret']
length = len(r)
x = np.array(r[0:length-1]).reshape(-1,1)
y = r[1:length]

linreg1 = LinearRegression()
linreg1.fit(x,y)
print([linreg1.intercept_,linreg1.coef_])
#MSE
print(np.sum((y-linreg1.predict(x))**2))

[0.009651786903953439, array([-0.05712743])]
0.4211772500075712


In [23]:
## Sentiment score only
s = np.array(tesla['sentiment'][0:length-1]).reshape(-1,1)

linreg2 = LinearRegression()
linreg2.fit(s,y)
print([linreg2.intercept_,linreg2.coef_])
#MSE
print(np.sum((y-linreg2.predict(s))**2))

[0.025550740048499408, array([-0.0901412])]
0.41365664361287885


In [24]:
## Lagged returns + Sentiment score
X = np.hstack([x,s])
linreg3 = LinearRegression()
linreg3.fit(X,y)
print([linreg3.intercept_,linreg3.coef_])
#MSE
print(np.sum((y-linreg3.predict(X))**2))

[0.0257894488608787, array([-0.04853911, -0.08915624])]
0.41315093662139957
